# Model Architecture

### Feature Extractors

In [7]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

N, n_H, n_W, n_C = 32, 28, 28, 3
n_filters = 5
k_size = 3
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

conv1 = Conv2D(filters=n_filters, kernel_size=k_size,
               padding='same', activation='relu')

max_pool1 = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

conv2 = Conv2D(filters=n_filters, kernel_size=k_size,
               padding='same', activation='relu')

max_pool2 = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

flatten = Flatten()


x = conv1(x)
W, B = conv1.get_weights()
print(f"x1: {x.shape}\n")
print(f"W1, B1: {W.shape}, {B.shape}\n")

x = max_pool1(x)
print(f"x1_pooling: {x.shape}\n")

x = conv2(x)
W, B = conv2.get_weights()
print(f"x2: {x.shape}\n")
print(f"W2, B2: {W.shape}, {B.shape}\n")

x = max_pool2(x)
print(f"x2_pooling: {x.shape}\n")

x = flatten(x)
print(f"x3: {x.shape}\n")


x1: (32, 28, 28, 5)

W1, B1: (3, 3, 3, 5), (5,)

x1_pooling: (32, 14, 14, 5)

x2: (32, 14, 14, 5)

W2, B2: (3, 3, 5, 5), (5,)

x2_pooling: (32, 7, 7, 5)

x3: (32, 245)



### Classifier

In [8]:
from tensorflow.keras.layers import Dense

n_denses = [50, 25, 10]

dense1 = Dense(units=n_denses[0], activation='relu')
dense2 = Dense(units=n_denses[1], activation='relu')
dense3 = Dense(units=n_denses[2], activation='softmax')

x = dense1(x)
W, B = dense1.get_weights()
print(f"dense1_x : {x.shape}")
print(f"dense1_W, B : {W.shape}, {B.shape}\n")

x = dense2(x)
W, B = dense2.get_weights()
print(f"dense2_x : {x.shape}")
print(f"dense2_W, B : {W.shape}, {B.shape}\n")

x = dense3(x)
W, B = dense3.get_weights()
print(f"dense3_x : {x.shape}")
print(f"dense3_W, B : {W.shape}, {B.shape}\n")


dense1_x : (32, 50)
dense1_W, B : (245, 50), (50,)

dense2_x : (32, 25)
dense2_W, B : (50, 25), (25,)

dense3_x : (32, 10)
dense3_W, B : (25, 10), (10,)



### Loss

In [10]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy

y = tf.random.uniform(shape=(32,),
                      minval=0, maxval=10,
                      dtype=tf.int32)

y_onehot = tf.one_hot(y, depth=10)

loss_cce_f = CategoricalCrossentropy()
loss_scce_f = SparseCategoricalCrossentropy()

loss_cce = loss_cce_f(y_onehot, x)
loss_scce = loss_scce_f(y, x)

print(f"loss_cce : {loss_cce}")
print(f"loss_scce : {loss_scce}")


loss_cce : 2.5526366233825684
loss_scce : 2.5526363849639893


# Model Implementation

### Implementation with Sequential Method

In [21]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))
print(f"x shape : {x.shape}\n")


model = Sequential()

model.add(Conv2D(filters=n_conv_neurons[0], kernel_size=k_size,
                 padding=padding, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[1], kernel_size=k_size,
                 padding=padding, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Conv2D(filters=n_conv_neurons[2], kernel_size=k_size,
                 padding=padding, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))

model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation='relu'))
model.add(Dense(units=n_dense_neurons[1], activation='relu'))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

prediction = model(x)
print(f"x shape : {x.shape}\n")
print(f"predictions : {prediction.shape}\n{prediction.numpy().squeeze()}\n")


x shape : (4, 28, 28, 3)

predictions : (4, 10)
[[0.10861005 0.09904233 0.17351937 0.09741263 0.06427553 0.08594694
  0.08502703 0.08651736 0.07387301 0.12577577]
 [0.12551346 0.138106   0.13230717 0.10824168 0.05766782 0.07866932
  0.08312546 0.08317037 0.07748305 0.11571573]
 [0.10101777 0.1101395  0.15174156 0.08803888 0.0661835  0.09291986
  0.09037475 0.07324909 0.07779466 0.14854048]
 [0.14770396 0.12621595 0.13379021 0.11599118 0.05057463 0.06956576
  0.07432584 0.08807235 0.07637641 0.11738367]]



### Implementation with Model Sub-classing

In [24]:
import tensorflow as tf

from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class TestCNN(Model):

  def __init__(self):
    super(TestCNN, self).__init__()

    self.conv1 = Conv2D(filters=n_conv_neurons[0], kernel_size=k_size,
                        padding=padding, activation='relu')
    self.conv1_maxpool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    self.conv2 = Conv2D(filters=n_conv_neurons[1], kernel_size=k_size,
                        padding=padding, activation='relu')
    self.conv2_maxpool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    self.conv3 = Conv2D(filters=n_conv_neurons[2], kernel_size=k_size,
                        padding=padding, activation='relu')
    self.conv3_maxpool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    self.flatten = Flatten()

    self.dense1 = Dense(units=n_dense_neurons[0], activation='relu')
    self.dense2 = Dense(units=n_dense_neurons[1], activation='relu')
    self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')

  def call(self, x):

    x = self.conv1(x)
    print(f"x (conv1) : {x.shape}\n")

    x = self.conv1_maxpool(x)
    print(f"x (maxpool1) : {x.shape}\n")

    x = self.conv2(x)
    print(f"x (conv2) : {x.shape}\n")

    x = self.conv2_maxpool(x)
    print(f"x (maxpool2) : {x.shape}\n")

    x = self.conv3(x)
    print(f"x (conv3) : {x.shape}\n")

    x = self.conv3_maxpool(x)
    print(f"x (maxpool3) : {x.shape}\n")

    x = self.flatten(x)
    print(f"x (flatten) : {x.shape}\n")

    x = self.dense1(x)
    print(f"x (dense1) : {x.shape}\n")

    x = self.dense2(x)
    print(f"x (dense2) : {x.shape}\n")

    x = self.dense3(x)
    print(f"x (dense3) : {x.shape}\n")

    return x


N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

model = TestCNN()
prediction = model(x)
print(f"Input shape : {x.shape}")
print(f"Output shape : {prediction.shape}\n{prediction.numpy().squeeze()}\n")

x (conv1) : (4, 28, 28, 10)

x (maxpool1) : (4, 14, 14, 10)

x (conv2) : (4, 14, 14, 20)

x (maxpool2) : (4, 7, 7, 20)

x (conv3) : (4, 7, 7, 30)

x (maxpool3) : (4, 3, 3, 30)

x (flatten) : (4, 270)

x (dense1) : (4, 50)

x (dense2) : (4, 30)

x (dense3) : (4, 10)

Input shape : (4, 28, 28, 3)
Output shape : (4, 10)
[[0.06154787 0.05784472 0.10824161 0.14004612 0.03349681 0.0779576
  0.07411264 0.26545987 0.10272696 0.07856585]
 [0.05214202 0.06106947 0.1157807  0.1742499  0.03788798 0.0920817
  0.06376361 0.19647142 0.10899609 0.09755717]
 [0.06348554 0.06559003 0.10020318 0.15306534 0.03758356 0.07381049
  0.07859762 0.22379984 0.10653437 0.09732997]
 [0.07173649 0.06137089 0.09990306 0.154537   0.03660833 0.07189689
  0.08722593 0.22956671 0.09233566 0.09481901]]



### Implementation with Layer sub-classing and Sequential Method

In [28]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class MyConv(Layer):

  def __init__(self, n_neurons):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neurons, kernel_size=3,
                       padding='same', activation='relu')
    self.maxpool = MaxPooling2D(pool_size=2, strides=2)

  def call(self, x):

    x = self.conv(x)
    x = self.maxpool(x)

    return x


class MyDense(Layer):

  def __init__(self, n_neurons, activation='relu'):
    super(MyDense, self).__init__()

    self.dense = Dense(units=n_neurons, activation=activation)

  def call(self, x):
    x = self.dense(x)

    return x

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

model = Sequential()

model.add(MyConv(n_conv_neurons[0]))
model.add(MyConv(n_conv_neurons[1]))
model.add(MyConv(n_conv_neurons[2]))
model.add(Flatten())

model.add(MyDense(n_dense_neurons[0]))
model.add(MyDense(n_dense_neurons[1]))
model.add(MyDense(n_dense_neurons[2], activation='softmax'))


prediction = model(x)
print(f"Input shape : {x.shape}\n")
print(f"Output shape : {prediction.shape}\n\n{prediction.numpy().squeeze()}\n")

Input shape : (4, 28, 28, 3)

Output shape : (4, 10)

[[0.06691208 0.07473395 0.10423584 0.10894565 0.09052815 0.12529768
  0.08357936 0.09136271 0.12320234 0.13120219]
 [0.06231073 0.07921544 0.10724083 0.11075092 0.1034582  0.12008381
  0.08113942 0.08812827 0.12613837 0.121534  ]
 [0.06475083 0.07038262 0.10253397 0.10397328 0.10273371 0.11361
  0.07831137 0.08073299 0.1425677  0.14040352]
 [0.06083912 0.07119374 0.10343128 0.10632354 0.10734161 0.113208
  0.07361764 0.09466471 0.1440629  0.12531738]]



### Implementation with Model and Layer Sub-classing

In [27]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense


class MyConv(Layer):

  def __init__(self, n_neurons):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neurons, kernel_size=3,
                       padding='same', activation='relu')
    self.maxpool = MaxPooling2D(pool_size=2, strides=2)

  def call(self, x):

    x = self.conv(x)
    x = self.maxpool(x)

    return x


class MyDense(Layer):

  def __init__(self, n_neurons, activation='relu'):
    super(MyDense, self).__init__()

    self.dense = Dense(units=n_neurons, activation=activation)

  def call(self, x):
    x = self.dense(x)

    return x


class MyModel(Model):

  def __init__(self):
    super(MyModel, self).__init__()

    self.fe = Sequential()
    self.fe.add(MyConv(n_conv_neurons[0]))
    self.fe.add(MyConv(n_conv_neurons[1]))
    self.fe.add(MyConv(n_conv_neurons[2]))
    self.fe.add(Flatten())

    self.classifier = Sequential()
    self.classifier.add(MyDense(n_dense_neurons[0]))
    self.classifier.add(MyDense(n_dense_neurons[1]))
    self.classifier.add(MyDense(n_dense_neurons[2], activation='softmax'))

  def call(self, x):

    x = self.fe(x)
    x = self.classifier(x)

    return x


N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2

x = tf.random.normal(shape=(N, n_H, n_W, n_C))

model = MyModel()
prediction = model(x)
print(f"Input shape : {x.shape}\n")
print(f"Output shape : {prediction.shape}\n\n{prediction.numpy().squeeze()}\n")

Input shape : (4, 28, 28, 3)

Output shape : (4, 10)

[[0.06975864 0.11626003 0.04827777 0.0689254  0.07566243 0.07666425
  0.06545237 0.11707094 0.12335395 0.23857427]
 [0.07166761 0.10977019 0.04609999 0.07103565 0.07215535 0.08096065
  0.06524555 0.10839176 0.10859492 0.26607823]
 [0.07065031 0.12157009 0.04345179 0.06451894 0.08151529 0.07486209
  0.07590704 0.1082351  0.10918756 0.25010175]
 [0.06267777 0.12054228 0.0400857  0.07542313 0.08263598 0.06822799
  0.06595408 0.11459569 0.11692706 0.2529303 ]]

